## Citi Bike NYC Data Engineering Project
--------------------------------------------------

### Project Overview

This project focuses on building a clean, reproducible data pipeline for Citi Bike rental data. The objective is to take raw trip-level and weather datasets, apply systematic data cleaning and normalization steps, and load the resulting tables into a relational SQLite database with analytical views.

- The notebook is designed with a data-engineering mindset rather than exploratory analysis.
- All transformations are intentional and aligned with a predefined SQL schema.

### Key goals of this project:

- Clean and standardize raw trip and weather data
- Normalize entities into dimension and fact tables
- Enforce schema consistency between pandas and SQLite
- Load validated data into a relational database for downstream analysis

### Business Questions

The project aims to create ready made sql views that support operational and strategic decision-making for the for the analytical team.

The following business questions guide the analytical views created in the database:

1. **Daily trip volume**
   - How many trips occur each day?
   - How does daily usage change over time?

2. **Peak usage hours**
   - At what hours of the day is demand highest?
   - Are peak hours consistent across days?

3. **Top stations**
   - Which stations generate the highest number of trip starts and ends?
   - Are there consistently high-performing stations?

4. **Station imbalance**
   - Which stations experience imbalances between departures and arrivals?
   - Where might bike rebalancing be required?

5. **User type behavior**
   - How do usage patterns differ between user types?
   - Are there differences in trip duration or time-of-day usage?

6. **Weather vs demand**
   - How does weather affect trip volume?
   - Which weather variables have the strongest relationship with demand?

##### Imports and Configurations

In [2]:
import pandas as pd
import sqlite3
from pathlib import Path

pd.set_option("display.max_columns", None)

CWD = Path.cwd()
DATADIR = CWD / "data"
DB_PATH = CWD / "db" / "citibike.db"
RIDE_Files = DATADIR.glob("JC-2016*.csv")
WEATHER_FILE = DATADIR / "newark_airport_2016.csv"

##### Load Raw Data

In [3]:
ride = pd.concat((pd.read_csv(file) for file in RIDE_Files), ignore_index=True)
weather = pd.read_csv(WEATHER_FILE)

##### Initial Inspection

In [4]:
ride.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247584 entries, 0 to 247583
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Trip Duration            247584 non-null  int64  
 1   Start Time               247584 non-null  object 
 2   Stop Time                247584 non-null  object 
 3   Start Station ID         247584 non-null  int64  
 4   Start Station Name       247584 non-null  object 
 5   Start Station Latitude   247584 non-null  float64
 6   Start Station Longitude  247584 non-null  float64
 7   End Station ID           247584 non-null  int64  
 8   End Station Name         247584 non-null  object 
 9   End Station Latitude     247584 non-null  float64
 10  End Station Longitude    247584 non-null  float64
 11  Bike ID                  247584 non-null  int64  
 12  User Type                247204 non-null  object 
 13  Birth Year               228585 non-null  float64
 14  Gend

In [5]:
ride.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,2
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,1
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0


In [6]:
ride.isna().sum()

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                    380
Birth Year                 18999
Gender                         0
dtype: int64

In [7]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   STATION  366 non-null    object 
 1   NAME     366 non-null    object 
 2   DATE     366 non-null    object 
 3   AWND     366 non-null    float64
 4   PGTM     0 non-null      float64
 5   PRCP     366 non-null    float64
 6   SNOW     366 non-null    float64
 7   SNWD     366 non-null    float64
 8   TAVG     366 non-null    int64  
 9   TMAX     366 non-null    int64  
 10  TMIN     366 non-null    int64  
 11  TSUN     0 non-null      float64
 12  WDF2     366 non-null    int64  
 13  WDF5     364 non-null    float64
 14  WSF2     366 non-null    float64
 15  WSF5     364 non-null    float64
dtypes: float64(9), int64(4), object(3)
memory usage: 45.9+ KB


In [8]:
weather.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
0,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-01,12.75,NaN,0.0,0.0,0.0,41,43,34,NaN,270,280.0,25.9,35.1
1,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-02,9.40,NaN,0.0,0.0,0.0,36,42,30,NaN,260,260.0,21.0,25.1
2,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-03,10.29,NaN,0.0,0.0,0.0,37,47,28,NaN,270,250.0,23.9,30.0
3,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-04,17.22,NaN,0.0,0.0,0.0,32,35,14,NaN,330,330.0,25.9,33.1
4,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-05,9.84,NaN,0.0,0.0,0.0,19,31,10,NaN,360,350.0,25.1,31.1


In [9]:
weather.isna().sum()

STATION      0
NAME         0
DATE         0
AWND         0
PGTM       366
PRCP         0
SNOW         0
SNWD         0
TAVG         0
TMAX         0
TMIN         0
TSUN       366
WDF2         0
WDF5         2
WSF2         0
WSF5         2
dtype: int64

##### Data Cleaning

##### Rename Columns (For Standardization Purposes)

In [13]:
ride.columns = (
    ride.columns
    .str.strip()
    .str.replace(" ", "_")
    .str.lower()
)

old_columns = weather.columns
new_columns = ["weather_station_id", "weather_station_name", "date", "avg_daily_wind_speed", \
               "peak_gust_time", "precipitation", "snowfall", "snow_depth", "avg_temp", "max_temp",\
                "min_temp", "daily_total_sushine", "fastest_2min_wind_dir", "fastest_5min_wind_dir", \
                "fastest_2min_wind_speed", "fastest_5min_wind_speed"]

weather.rename(columns=dict(zip(old_columns, new_columns)), inplace=True)

##### Data Type Conversion

In [14]:
ride["start_time"] = pd.to_datetime(ride["start_time"], errors="coerce")
ride["stop_time"] = pd.to_datetime(ride["stop_time"], errors="coerce")
ride["trip_duration"] = pd.to_numeric(ride["trip_duration"], errors="coerce")
ride["birth_year"] = pd.to_numeric(ride["birth_year"], errors="coerce")\
    .astype("Int64")

weather["date"] = pd.to_datetime(weather["date"], errors="coerce")

##### Drop Null

In [15]:
# Drop rows missing essential trip identifiers
ride = ride.dropna(subset=[
    "start_time",
    "start_station_id",
    "bike_id"
])

In [16]:
# Drop null columns in the weather dataframe
weather = weather.drop(columns=["peak_gust_time", "daily_total_sushine"], axis=1)

##### Standardize Categorical Text

In [17]:
# Lowercase and remove extra space if any
ride["user_type"] = ride["user_type"].str.lower().str.strip()
ride["gender"] = ride["gender"].astype(str).str.lower().str.strip()

##### Extract Stations' Data

In [18]:
start_stations = ride[[
    "start_station_id",
    "start_station_name",
    "start_station_longitude",
    "start_station_latitude"
    ]].rename(columns={
        "start_station_id": "station_id",
        "start_station_name": "station_name",
        "start_station_longitude": "station_longitude",
        "start_station_latitude": "station_latitude"
})

end_stations = ride[[
    "end_station_id",
    "end_station_name",
    "end_station_longitude",
    "end_station_latitude"
    ]].rename(columns={
        "end_station_id": "station_id",
        "end_station_name": "station_name",
        "end_station_longitude": "station_longitude",
        "end_station_latitude": "station_latitude"
})

stations = (
    pd.concat([start_stations, end_stations])
    .dropna(subset=["station_id"])
    .drop_duplicates(subset=["station_id"])
    .reset_index(drop=True)
)

##### Extract Trips' Data

In [19]:
trips = ride[[
    "trip_duration",
    "start_time",
    "stop_time",
    "start_station_id",
    "end_station_id",
    "bike_id",
    "user_type",
    "birth_year",
    "gender"
]].copy()

trips.insert(0, "trip_id", range(1, len(trips) + 1))

##### Extract Weather Stations' Data

In [20]:
weather_station = weather[[
    "weather_station_id",
    "weather_station_name"
]].drop_duplicates()

##### Extract Weather's Data

In [21]:
weather_data = weather.drop(columns=["weather_station_name"])

##### Database Set Up

##### Schema Validation

In [22]:
def validate_schema(df, expected_cols, table_name):

    if set(df.columns) != set(expected_cols):
        raise ValueError(f"Schema mismatch for table '{table_name}'")


validate_schema(stations, ["station_id", "station_name", "station_longitude", \
                           "station_latitude"], "stations")


validate_schema(trips, ["trip_id", "trip_duration", "start_time", "stop_time", \
                        "start_station_id", "end_station_id", "bike_id", \
                        "user_type", "birth_year", "gender"], "trips")

##### Load Data Into SQLITE

The database and the tables can be created by running the [setupdb.py](.\scripts\setupdb.py) file once or importing the *init_db()* function from it and call it once.

In [26]:
from scripts.setupdb import init_db
init_db()

The following cell code should run once to avoid inserting duplicate data into the tables or we can change the parameter *if_exists* to *replace*.

In [ ]:
""" conn = sqlite3.connect(DB_PATH)


stations.to_sql("stations", conn, if_exists="append", index=False)
trips.to_sql("trips", conn, if_exists="append", index=False)
weather_station.to_sql("weather_station", conn, if_exists="append", index=False)
weather_data.to_sql("weather_data", conn, if_exists="append", index=False)


conn.close() """

##### Creating Analytical Views

Each view has a sql file that creates it. The file [build_views.py](.\scripts/build_views.py) is responsible to build the views. Either run the file or import the function.

##### Exporting Views' Data

In [35]:
from scripts.export_views import export_views
#export_views()